In [0]:
# mount google drive and download dataset from google drive

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=a99319ac802545a236944967e362bca8f82ed87450fb4be878f45161dd89293b
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.1 GB  | Proc size: 1.1 GB
GPU RAM Free: 7408MB | Used: 203MB | Util   3% | Total 7611MB


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# 1Z8pft8C4Bu87u7Z4Vf4hNzE5TKthAlIl -- id taken from the shareable link of the zipped folder 
downloaded = drive.CreateFile({'id':"13BzrZ7w9Yt9PE7NnPhtJAFH2zDCirkbb"})   # 14 mushrooms
print('title: %s, mimeType: %s' % (downloaded['title'], downloaded['mimeType']))
downloaded.GetContentFile('image1.tgz')

title: 400.zip, mimeType: application/x-zip-compressed


In [0]:
from zipfile import ZipFile
file_name = "image1.tgz"

with ZipFile(file_name, 'r') as zip:
  zip.extractall("~/Content/train_data")
  print('Done')

Done


In [0]:
# making the model

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 # to install this: pip install opencv-python

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [0]:
try:

    import tensorflow as tf
    import cv2
    import os
    import pickle
    import numpy as np
    print("Library Loaded Successfully ..........")
except:
    print("Library not Found ! ")


class MasterImage(object):

    def __init__(self,PATH='', IMAGE_SIZE = 50):
        self.PATH = PATH
        self.IMAGE_SIZE = IMAGE_SIZE

        self.image_data = []
        self.x_data = []
        self.y_data = []
        self.CATEGORIES = []

        # This will get List of categories
        self.list_categories = []

    def get_categories(self):
        for path in os.listdir(self.PATH):
            if '.DS_Store' in path:
                pass
            else:
                self.list_categories.append(path)
        print("Found Categories ",self.list_categories,'\n')
        return self.list_categories

    def Process_Image(self):
        try:
            """
            Return Numpy array of image
            :return: X_Data, Y_Data
            """
            self.CATEGORIES = self.get_categories()
            for categories in self.CATEGORIES:                                                  # Iterate over categories

                train_folder_path = os.path.join(self.PATH, categories)                         # Folder Path
                class_index = self.CATEGORIES.index(categories)                                 # this will get index for classification

                for img in os.listdir(train_folder_path):                                       # This will iterate in the Folder
                    new_path = os.path.join(train_folder_path, img)                             # image Path

                    try:        # if any image is corrupted
                        image_data_temp = cv2.imread(new_path,cv2.IMREAD_GRAYSCALE)                 # Read Image as numbers
                        image_temp_resize = cv2.resize(image_data_temp,(self.IMAGE_SIZE,self.IMAGE_SIZE))
                        self.image_data.append([image_temp_resize,class_index])
                    except:
                        pass

            data = np.asanyarray(self.image_data)

            # Iterate over the Data
            for x in data:
                self.x_data.append(x[0])        # Get the X_Data
                self.y_data.append(x[1])        # get the label

            X_Data = np.asarray(self.x_data) / (255.0)      # Normalize Data
            Y_Data = np.asarray(self.y_data)

            # reshape x_Data

            X_Data = X_Data.reshape(-1, self.IMAGE_SIZE, self.IMAGE_SIZE, 1)

            return X_Data, Y_Data
        except:
            print("Failed to run Function Process Image ")

    def pickle_image(self):

        """
        :return: None Creates a Pickle Object of DataSet
        """
        # Call the Function and Get the Data
        X_Data,Y_Data = self.Process_Image()

        # Write the Entire Data into a Pickle File
        pickle_out = open('X_Data','wb')
        pickle.dump(X_Data, pickle_out, protocol = 4)
        pickle_out.close()

        # Write the Y Label Data
        pickle_out = open('Y_Data', 'wb')
        pickle.dump(Y_Data, pickle_out, protocol = 4)
        pickle_out.close()

        print("Pickled Image Successfully ")
        return X_Data,Y_Data

    def load_dataset(self):

        try:
            # Read the Data from Pickle Object
            X_Temp = open('X_Data','rb')
            X_Data = pickle.load(X_Temp)

            Y_Temp = open('Y_Data','rb')
            Y_Data = pickle.load(Y_Temp)

            print('Reading Dataset from PIckle Object')

            return X_Data,Y_Data

        except:
            print('Could not Found Pickle File ')
            print('Loading File and Dataset  ..........')

            X_Data,Y_Data = self.pickle_image()
            return X_Data,Y_Data

Library Loaded Successfully ..........


In [0]:
if __name__ == "__main__":
#     path = '/Users/soumilshah/IdeaProjects/mytensorflow/Dataset/training_set'
    path = '~/Content/train_data'
    a = MasterImage(PATH=path, IMAGE_SIZE=80)

    X_Data,Y_Data = a.load_dataset()
    print(X_Data.shape)

Could not Found Pickle File 
Loading File and Dataset  ..........
Found Categories  ['14044_Flammulaster_granulosus', '14181_Geastrum_pectinatum', '10052_Agaricus_altipes', '13330_Cinereomyces_lindbladii', '10234_Amanita_fulva', '14184_Geastrum_rufescens', '10056_Agaricus_arvensis', '10666_Ascocoryne_cylichnium', '13459_Encoelia_furfuracea', '10317_Amylostereum_chailletii', '12226_Cordyceps_militaris', '11573_Chalciporus_piperatus', '11912_Conocybe_albipes', '10108_Agaricus_sylvicola', '12763_Hericium_cirrhatum', '11284_Calyptella_campanula', '10065_Agaricus_campestris', '12392_Cortinarius_danicus', '14051_Flammulaster_subincarnatus', '12399_Cortinarius_diasemospermus var. diasemospermus_', '11064_Imleria_badia', '11083_Boletus_pinophilus', '13798_Entoloma_vernum', '12076_Coprinopsis_ephemeroides', '10526_Armillaria_mellea', '14148_Galerina_vittiformis', '14428_Gymnopilus_penetrans', '13528_Entoloma_atrocoeruleum', '10079_Agaricus_impudicus', '10263_Amanita_rubescens var. rubescens', '

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_Data, Y_Data, test_size=0.2)

In [0]:
# X_Data=X_Data/255

model = Sequential()

model.add(   Conv2D(150, (3,3), input_shape = X_train.shape[1:])   )
# model.add(   Conv2D(150, (3,3), input_shape = X.shape[1:])   )
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.4))
model.add(Conv2D(75, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dense(128))

model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(64))

model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(64))

model.add(Dropout(0.4))
model.add(Dense(256))
model.add(Activation('softmax'))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(X_Data, Y_Data, batch_size=64, epochs=20, validation_data=(X_test, y_test))



Epoch 1/20
413/413 [==============================] - 28s 69ms/step - loss: nan - accuracy: 5.3060e-04 - val_loss: nan - val_accuracy: 7.5801e-04
Epoch 2/20
413/413 [==============================] - 28s 68ms/step - loss: nan - accuracy: 4.9270e-04 - val_loss: nan - val_accuracy: 7.5801e-04
Epoch 3/20
413/413 [==============================] - 28s 68ms/step - loss: nan - accuracy: 4.9270e-04 - val_loss: nan - val_accuracy: 7.5801e-04
Epoch 4/20
413/413 [==============================] - 28s 69ms/step - loss: nan - accuracy: 4.9270e-04 - val_loss: nan - val_accuracy: 7.5801e-04
Epoch 5/20
413/413 [==============================] - 28s 68ms/step - loss: nan - accuracy: 4.9270e-04 - val_loss: nan - val_accuracy: 7.5801e-04
Epoch 6/20
413/413 [==============================] - 28s 68ms/step - loss: nan - accuracy: 4.9270e-04 - val_loss: nan - val_accuracy: 7.5801e-04
Epoch 7/20
413/413 [==============================] - 28s 68ms/step - loss: nan - accuracy: 4.9270e-04 - val_loss: nan - val

KeyboardInterrupt: ignored

In [0]:
# batch size kinda depends on the size of the dataset, 32 is being used for now for the test, might change it up to 200 or 2000 during implementation

In [0]:
# serializeing model to JSON

In [0]:
from keras.models import model_from_json

Using TensorFlow backend.


In [0]:
scores = model.evaluate(X_train, y_train, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 62.29%


In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)

In [0]:
# load json and create model
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [0]:
# tf save model

In [0]:
model.save('462shroom.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 462shroom.model/assets


In [0]:
def prepare(filepath):
  img_size=50
  img_array=cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
  new_array=cv2.resize(img_array, (img_size, img_size))
  return new_array.reshapt(-1, img_size, img_size, 1)

In [0]:
model = tf.keras.models.load_model('462shroom.model')

In [0]:
# test image folder id: 1w7VHZFnmiu0rhNuWT5h5JV1UJAeIlmM4
# image id: 17AShz0ODQrSDdFKrxi4r6nKjG6KEsNKm

test_img = drive.CreateFile({'id':"17AShz0ODQrSDdFKrxi4r6nKjG6KEsNKm"})   # 14 mushrooms
print('title: %s, mimeType: %s' % (test_img['title'], test_img['mimeType']))
test_img.GetContentFile('test.jpg')


title: BWP2013PIC11802314.JPG, mimeType: image/jpeg


In [0]:
print(test_img)

GoogleDriveFile({'id': '17AShz0ODQrSDdFKrxi4r6nKjG6KEsNKm', 'kind': 'drive#file', 'etag': '"StLURADGvFAD7O9Kbwy7cPbE-UU/MTU4NjYzNDM5NzM0MA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/17AShz0ODQrSDdFKrxi4r6nKjG6KEsNKm', 'webContentLink': 'https://drive.google.com/a/csu.fullerton.edu/uc?id=17AShz0ODQrSDdFKrxi4r6nKjG6KEsNKm&export=download', 'alternateLink': 'https://drive.google.com/a/csu.fullerton.edu/file/d/17AShz0ODQrSDdFKrxi4r6nKjG6KEsNKm/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/a/csu.fullerton.edu/file/d/17AShz0ODQrSDdFKrxi4r6nKjG6KEsNKm/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/image/jpeg', 'thumbnailLink': 'https://lh3.googleusercontent.com/EKVPSAk8MF88UB35PdYkCiCzfTrPqNeU6BANXX2ngDdMdIJcZgSZ7xNTKghd79HMqGFgtP2r5hU=s220', 'title': 'BWP2013PIC11802314.JPG', 'mimeType': 'image/jpeg', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriter

In [0]:
file_name = "test.tgz"

with ZipFile(file_name, 'r') as zip:
  zip.extractall("~/Content/test_images")
  print('Done')

In [0]:
prediction = model.predict([prepare(test_img)])

SystemError: ignored

In [0]:
model.save('path_to_saved_model', save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: path_to_saved_model/assets
